# Synchronize folder-base data

In [12]:
# install the data_converter module which allows to convert a text file to a hdf5 file
# !pip install -e ./data_converter

# Create sample data in a folder structure

In [16]:
from qdrive.dataset import generate_dataset_info
from data_converter.converter_txt_to_hdf5 import TxtToHdf5Converter
from datetime import datetime, timedelta
import os
import json
import numpy as np
import random

data_path = "data/folder_base_data/"
sample_names = ["sample_A", "sample_B", "sample_C"]
sample_name = random.choice(sample_names)

# Create measurement data (measurement.txt & metadata.json file)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
path = f"{data_path}/{sample_name}/{timestamp}/"
os.makedirs(path, exist_ok=True)

t1_times = np.linspace(0, 100, 50)  # microseconds
t1_signal = np.exp(-t1_times/25) + 0.02 * np.random.randn(50)  # T1 = 25 μs
np.savetxt(f"{path}measurement.txt", np.column_stack([t1_times, t1_signal]), 
           header="Time(us)\tSignal(mV)", delimiter="\t", comments="")

metadata = {
    "measurement_type": "T1_decay",
    "qubit_id": "Q1",
    "temperature(K)": 0.015,
    "power(dBm)": -20,
    "frequency(Hz)": 5.2e9,
    "fitted_T1(us)": 25.3,
}

with open(f"{path}metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)


# Generate _QH_dataset_info.yaml manifest file
# The converter TxtToHdf5Converter is used to convert the measurement.txt file to a hdf5 file
generate_dataset_info(path,
    dataset_name = "T1_decay_measurement",
    creation = datetime.now(),
    description="T1 decay measurement for qubit relaxation time characterization.",
    attributes={"sample" : sample_name, "qubit_id": "Q1", "measurement_type": "T1_decay"},
    keywords=["T1", "decay", "relaxation", "characterization"],
    converters = [TxtToHdf5Converter],
    # skip=["*.json", "raw_data/*"]
)

# Add folder to sync sources

In [ ]:
from etiket_client.sync.backends.sources import add_sync_source
from etiket_client.python_api.scopes import get_selected_scope
from etiket_client.sync.backends.filebase.filebase_sync_class import FileBaseSync
from etiket_client.sync.backends.filebase.filebase_config_class import FileBaseConfigData
import qdrive

from pathlib import Path
data_path = Path("./data/folder_base_data/")
config = FileBaseConfigData(root_directory=data_path, server_folder = False)
add_sync_source('folder base test', FileBaseSync, config, get_selected_scope())
qdrive.restart_sync_agent()


### Once the source is added:
- check the new data appear in DataQruiser
- create more datasets
- add new files to an existing dataset folder
- modify the attributes in the __QH_dataset_info.yaml file